In [1]:
workspace()
using Bolas
using Derivatives
using BolaIntervalo


    ^(Derive,Real) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Derivatives.jl:93
is ambiguous with: 
    ^(Any,Integer) at intfuncs.jl:86.
To fix, define 
    ^(Derive,Integer)
before the new definition.
    promote_type(Type{T<:Number},Type{Interval}) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Intervals.jl:467
is ambiguous with: 
    promote_type(Type{None},Type{T}) at promotion.jl:103.
To fix, define 
    promote_type(Type{None},Type{Interval})
before the new definition.
    promote_type(Type{T<:Number},Type{Interval}) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Intervals.jl:467
is ambiguous with: 
    promote_type(Type{T},Type{T}) at promotion.jl:101.
To fix, define 
    promote_type(Type{_<:Number},Type{_<:Number})
before the new definition.


In [26]:
A=Bola(4,1)
B=Bola(2,1)
N=Bola(0.9999942779541016,9.5367431640625e-6)

bola_a_intervalo(A)

Interval(3.0,5.0)

In [18]:
function revisar_cercanas(A,B,tol) #revisa si una raíz dada está cerca de las demás
    
    for i = 1:length(B)
        
        if distancia(A,B[i]) <= tol
            return(true) 
        end
        
        
    end
    
    return(false)
end

function quitar_repetidas(A,tol) #remueve raices que esten mas cerca que una cierta tolerancia
   
    temp=Bola[]
    
    for i=1:length(A)
        
        if !(revisar_cercanas(A[i],temp,tol))
            push!(temp,A[i])
        end
        
    end
    
    return(temp)
end

function quitar_no_deseadas(F::Function,B,x) #Funcion para quitar bolas donde la derivada de la funcion se hace cero
    
    temp=Bola[]
    dF(x)=F(makex(x)).d
    
    for i=1:length(B)
        if !contiene(dF(B[i]),x)
            push!(temp,B[i])
        end
    end
    
    return(temp)
end

function quitar_no_utiles(F::Function,B,x) #Se encarga de quitar bolas que no contienen ceros de la funcion
    
    temp=Bola[]
    
    for i=1:length(B)
        if contiene(F(B[i]),x)
            push!(temp,B[i])
        end
    end
    
    return(temp)
end



function operador_de_newton_bola(F::Function,B::Bola) 
    
    dF(x)=F(makex(x)).d
    m=Bola(B.centro,1e-15) #Le asigno un radio mínimo al punto m 
    
    
    return (m-F(m)/(dF(B)))
    
end   

function metodo_newton_bola(F::Function, B::Bola)
    
    X=[B]
    
    i=0
    tolbis=1e-3
    tolradio=1e-15
    tolsimilar = 1e-2
    
    while(X[1].radio>tolbis)
        
       X=bisectar_bolas(X)
       X=quitar_no_utiles(F,X,0)
    
        
        i+=1
        
        if i==50
            break
        end
        
        X
 
    end
    
    X=quitar_no_deseadas(F,X,0)
    
    j=0
    
    radio_promedio=1
    
    while(tolradio<radio_promedio)
        
        radio_promedio=0
        
        
        for i=1:length(X)
            
            temp=operador_de_newton_bola(F,X[i])
            
            if interseccion_de_bolas(X[i],temp)==nothing
                X[i]=interseccion_de_bolas(B,temp)
            else
                X[i]=interseccion_de_bolas(X[i],temp)
            end
            
            radio_promedio+=(X[i].radio)
        end
         
        radio_promedio/=length(X)
        
        j+=1
        
        if j==50
            break
        end
    end
    
    X=quitar_repetidas(X,tolsimilar)
    
    for i=1:length(X) 
        println("Posible raíz en $(X[i].centro) ± $(X[i].radio)")
    end
    
end

f(x)=(x-1)*(x-2)*(x-3)*(x-4)*(x-5)*(x-6)
g(x)=(x-2)*(x+4)*(exp(x)-1)*exp(x)
M=metodo_newton_bola(f,Bola(1,30))

Posible raíz en 0.99908447265625 ± 0.00091552734375
Posible raíz en 2.00067138671875 ± 0.00091552734375
Posible raíz en 3.0 ± 1.3322676295501878e-15
Posible raíz en 4.000000000000001 ± 1.7763568394002505e-15
Posible raíz en 5.0 ± 2.0000000000073476e-15
Posible raíz en 5.99969482421875 ± 0.00091552734375


In [20]:
4-4.000000000000001

-8.881784197001252e-16

In [82]:
g(2.471156143761765985091067496914777668228857824447285685081296242485736952738683e+00-3.921311541886343194711952615628528423130597924445861642547724252737956805218929e-17)

-5.849978886418583e-6

In [86]:
operador_de_newton_bola(f,Bola(3.0056896291766835,0.5880009079680009))

LoadError: No se puede dividir por una bola que contine el 0
while loading In[86], in expression starting on line 1

In [3]:
M=[1,2,3,4,5,6,6,6,7,1]


for i=1:length(M)
    
    temp=Int[]
    
    for i=1:length(M)
        
        if !(M[i] in temp)
            push!(temp,M[i])
        end
    end
        
    return(temp)
end



7-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6
 7

In [2]:


quitar_repetidas(M,.05)

LoadError: M not defined
while loading In[2], in expression starting on line 30

In [62]:
M=[Bola(3),Bola(2),Bola(3),Bola(4),Bola(2),Bola(3.1)]

6-element Array{Bola,1}:
 Bola(3,1)  
 Bola(2,1)  
 Bola(3,1)  
 Bola(4,1)  
 Bola(2,1)  
 Bola(3.1,1)

In [50]:
function quitar_bolas_similares(B,tol::Real)
    
    return(temp)
end

quitar_bolas_similares (generic function with 1 method)

false

In [54]:
sort(M)

LoadError: `isless` has no method matching isless(::Bola, ::Bola)
while loading In[54], in expression starting on line 1

In [47]:
57847/1.16

49868.10344827586

In [48]:
150000/15

10000.0

In [106]:
M=Bola([1,1],1)
N=Bola([1,2],1)

Bola([1,2],1)

In [110]:
M-N

Bola([0,-1],2)

In [9]:
A=Bola(2,1)
B=Bola(3,2)

A/B

Bola(0.6666666666666666,6.333333333333334)

In [10]:
using Intervals

In [11]:
Interval(1,3)/Interval(1,5)

Interval(0.19999999999999998,3.0)